<a href="https://colab.research.google.com/github/QColeman97/AudioTagger/blob/master/TagAudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns # for data visualization

# import IPython
# import IPython.display as ipd #To play sound in notebook
# import scipy as sci
# import wave
from pathlib import Path

from scipy.fftpack import fft #Fast Fourier Transformation
from scipy.io import wavfile

import librosa
from librosa import display
import os
import glob

input_path = 'drive/My Drive/AudioTaggerData/'


# input_path = '../AudioTaggerData/'
train_files_path = input_path + 'FSDKaggle2018.audio_train'
test_files_path = input_path + 'FSDKaggle2018.audio_test'
train_csv_path = (input_path +
                  'FSDKaggle2018.meta/train_post_competition.csv')
test_csv_path = (input_path +
                 'FSDKaggle2018.meta/' +
                 'test_post_competition_scoring_clips.csv')

#scipy.wavfile.read returns rate of wave, and # of data read
filename = '/001ca53d.wav'
# sample_rate, samples = wavfile.read(str(audio_train_file) + filename)
# print(samples)
# print(train.shape)

# Data preprocessing part

df_train = pd.read_csv(train_csv_path)
df_test = pd.read_csv(test_csv_path)

unique_labels = df_train.label.unique()
num_class = len(unique_labels)
label2index = {label: index for index, label in enumerate(unique_labels)}

# print('Label to index:')
# print(label2index)

train_dict = pd.Series.from_csv(train_csv_path, header = 0).to_dict()
test_dict = pd.Series.from_csv(test_csv_path, header = 0).to_dict()


# print('Train dict at:')
# print(test_dict['40f595b9.wav'])
# KeyError: '40f595b9.wav'

# train_df = pd.read_csv(train_csv_path, header = 0)
# train_dict = train_df.transpose().to_dict()
# print('train dict:')
# print(train_dict)

#Indices of manually verified training data
# verified_train = np.array(df_train[df_train.manually_verified == 1].index)
# print(len(verified_train))
# print(len(df_train))

#array of labels in number form (0 = hi-hat, 1 = saxophone, etc)
label_emb_indices = np.array([label2index[label] for label in df_train.label])

label_emb_test_indices = np.array([label2index[label] for label in df_test.label])

# print(plain_y_train)

# print('Label emb indices:')
# print(label_emb_indices)

# # Approach X uses longer sound, then it uses suppressed
# # confX['sampling_rate'] = 26000
# # sampling_rate = 44100 # Original file sr
# sampling_rate = 32000
# # duration = 4
# duration = 5
# # confX['hop_length'] = 520  # 20ms
# hop_length = 192
# fmin = 20
# fmax = sampling_rate // 2
# # confX['n_mels'] = 48
# n_mels = 128
# # confX['n_fft'] = confX['n_mels'] * 20
# n_fft = 1024
# audio_split = 'dont_crop'
# samples = sampling_rate * duration
# dims = (n_mels, 1 + int(np.floor(samples / hop_length)), 1)


def pre_process(pathname):
    sampling_rate = 32000
    # duration = 4
    # duration = 5
    # confX['hop_length'] = 520  # 20ms
    hop_length = 192
    # fmin = 20
    # fmax = sampling_rate // 2
    fmax = None
    # confX['n_mels'] = 48
    n_mels = 128 # originally
#     n_mels = 32
    # confX['n_fft'] = confX['n_mels'] * 20
    n_fft = 1024
    # audio_split = 'dont_crop'
    # samples = sampling_rate * duration
    # dims = (n_mels, 1 + int(np.floor(samples / hop_length)), 1)

    # TODO: Correct spectrogram creation to minimize bad data

    # y, sr = librosa.load(pathname, sr = None)
    y, sr = librosa.load(pathname, sr = sampling_rate)
    # print('Y len:', len(y))
    y, (trim_begin, trim_end) = librosa.effects.trim(y)

    # print('Y len:', len(y))
#     y = librosa.effects.time_stretch(y, 2)
#     print('Y len:', len(y))

    # while len(y) > 10000:
    #     y = librosa.effects.time_stretch(y, 2)
    #     print('Y len:', len(y))
    # print()

    # Amplitudes of STFT
    stft = np.abs(librosa.stft(y, n_fft = n_fft, hop_length = hop_length,
                               window = 'hann', center = True,
                               pad_mode = 'reflect'))

    # print('stft shape:', stft.shape)

    freqs = librosa.core.fft_frequencies(sr = sampling_rate, n_fft = n_fft)
    stft = librosa.perceptual_weighting(stft*2, freqs, ref = 1.0, amin = 1e-10,
                                        top_db = 99.0)

    # print('stft shape:', stft.shape)

    # Apply mel filterbank
    # Power param is set to 2 (power) by default
    mel_spect = librosa.feature.melspectrogram(S = stft, sr = sampling_rate,
                                               n_mels = n_mels, fmax = fmax)

    # print('mel shape:', mel_spect.shape)

    log_mel_spect = librosa.core.power_to_db(mel_spect)

    # print('log mel shape:', log_mel_spect.shape)

    # spectrogram = librosa.feature.melspectrogram(S = stft)
    # Keep spectrogram
    # return np.asarray(spectrogram)
    return np.asarray(log_mel_spect)


# pre_process(audio_train_file + filename)
def get_data(pathname, training = True):
    file_list = glob.glob(os.path.join(pathname, '*.wav'))

    if training:
        data_f = open('Audio.train', 'w')
    else:
        data_f = open('Audio.test', 'w')

    # print(file_list)
#     spectrograms, times = [], []
    spectrograms = np.ndarray((9474, 256, 128))

    for i, file in enumerate(file_list):
        print("%04d / %d | %s" % (i + 1, len(file_list), file))

        try:
            spectrogram = pre_process(file)
        except Exception:
            print('Weird, couldnt convert to spectrogram, skipping file')
            continue

        # times.append(spectrogram.shape[1])

        # Originally 2000 (average)
#         time_restriction = 64
        time_restriction = 256
        if time_restriction >= spectrogram.shape[1]:
            pad_amount = time_restriction - spectrogram.shape[1]
            # Use avg or max time
            spectrogram = np.pad(spectrogram, ((0, 0), (0, pad_amount)),
                                 'minimum')
        else:
            spectrogram = spectrogram[:, :time_restriction]

        spectrogram = spectrogram.transpose()

        # print("Spectrogram Shape:", spectrogram.shape)

#         spectrograms.append(spectrogram.astype(np.float32))
        for j in range(len(spectrogram)):
            for k in range(len(spectrogram[j])):
                spectrograms[i][j][k] = spectrogram[j][k].astype(np.float32)
                
        # data_f.write(np.array2string(spectrogram) + '\n\n')
#         np.savetxt(data_f, spectrogram)
#         data_f.write('\n')

        # if i > 500:
        #     break
        # if 32 < i < 50:  # 34 is a weird one
#         if i % 12 == 0:
        
#             plt.figure("General-Purpose ")
#             plt.clf()
#             plt.subplots_adjust(right = 0.98, left = 0.1, bottom = 0.1,
#                                 top = 0.99)
#             plt.imshow(spectrogram, origin = "lower",
#                        interpolation = "nearest", cmap = "viridis")
#             plt.xlabel("%d bins" % spectrogram.shape[1])
#             plt.ylabel("%d frames" % spectrogram.shape[0])
#             plt.colorbar()
#             plt.show()
        
            # display.specshow(spectrogram, y_axis = 'log', x_axis = 'time')
            #
            # plt.title('Mel Spectrogram')
            # plt.colorbar(format = '%+2.0f dB')
            # plt.tight_layout()
            # plt.show()
        
            # print('Spectrogram:', i)
            # print(spectrogram)

    # average_time = np.average(times)
    # print('Average timesteps:', average_time)
    # max_time = np.amax(times)
    # print('Max timesteps:', max_time)

    return spectrograms
#     data_f.close()


def get_labels(pathname, training = True):
    file_list = glob.glob(os.path.join(pathname, '*.wav'))

    if training:
        labels_f = open('Labels.train', 'w')
    else:
        labels_f = open('Labels.test', 'w')

#     labels = []
    labels = np.ndarray((1570, 41))
    for i, file in enumerate(file_list):
#         label = np.zeros((41,))
        categ = (train_dict[file.split('/')[-1]] if
            (training) else test_dict[file.split('/')[-1]])
        hot_index = label2index[categ]
        labels[i][hot_index] = 1
#         labels.append(label)
#         labels_f.write(' '.join([str(x) for x in label]) + '\n')

    return np.array(labels)
#     labels_f.close()


# get_data(train_files_path, training = True)
# get_labels(train_files_path, training = True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4141: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


In [0]:
from keras import models, layers
from keras_preprocessing.image import ImageDataGenerator
# import librosa
# from librosa import display
import numpy as np
# import matplotlib.pyplot as plt
# import IPython.display as ipd  # only for IPython notebooks
# import pyaudio
# import wave
# import PreProcess


# #1 preprocess method
# Normalize input w/ batch normalization (BN)
# Input is trnasformed into logmel domain (turned into log-mel spectrograms)
# BN again & reshaped (time, frequency, 1) - greyscale image
# Then conv stuff

# Hyperparams / config to test:
# ch 7: (advanced)
#   model ensembling (of 2D CNN & Combined 1D CNN & RNN combo, etc??)
#   Replace Conv2D w/ SeparableConv2D
#   Batch normailization (can replace regularization)
#   Inception
#   Residual connections
#   DenseNet is a good model to follow (top leader) or fine tune from

# ch 5: (convnets)
#   Tune HP (# neurons, layers, epochs, batch_size)
#   dropout, regularization (can be replaced by BN)
#   Data augmentation

# K-Fold Validation


# class LyrParams:
#     # Describes, for a layer, # channels/neurons, dropout rate &
#     # regularization choice (Lyr = layer)
#     def __init__(self, units, dpt = None):
#         self.units = units
#         self.dpt = dpt

# nn.add(layers.Conv2D(prm_lyrs[1].units, (3, 3), activation = 'relu'))
# if prm_lyrs[0].dpt:
#   nn.add(layers.dpt(prm_lyrs[0].dpt))

# Lab 3 is a good one for backup
# Model structure after best methods - 6 conv units (conv *2 + maxpool)
def make_model(input_shape):
    # Current Shape: (500, 128, 1)  # 500 = timesteps, 128 = frequencies
    nn = models.Sequential()
    nn.add(layers.SeparableConv2D(64, (3, 3), padding = 'same',
                                  activation = 'relu',
                                  input_shape = input_shape))
    # Shape: (126, 498, 64)
    nn.add(layers.BatchNormalization())
    nn.add(layers.SeparableConv2D(64, (3, 3), padding = 'same',
                                  activation = 'relu'))
    # Shape: (124, 496, 64)
    nn.add(layers.BatchNormalization())
    nn.add(layers.MaxPooling2D((2, 2)))
    # Shape: (62, 248, 64)
    nn.add(layers.Dropout(0.3))

    nn.add(layers.SeparableConv2D(128, (3, 3), padding = 'same',
                                  activation = 'relu'))
    # Shape: (60, 246, 128)
    nn.add(layers.BatchNormalization())
    nn.add(layers.SeparableConv2D(128, (3, 3), padding = 'same',
                                  activation = 'relu'))
    # Shape: (58, 244, 128)
    nn.add(layers.BatchNormalization())
    nn.add(layers.MaxPooling2D((2, 2)))
    # Shape: (29, 122, 128)
    nn.add(layers.Dropout(0.3))

#     # Possibly make this block more like the others
#     nn.add(layers.SeparableConv2D(256, (3, 3), padding = 'same',
#                                   activation = 'relu'))
#     # Shape: (27, 120, 256)
#     nn.add(layers.BatchNormalization())
#     nn.add(layers.Dropout(0.3))
    nn.add(layers.SeparableConv2D(256, (3, 3), padding = 'same',
                                  activation = 'relu'))
    # Shape: (25, 118, 256)
    nn.add(layers.BatchNormalization())
    nn.add(layers.Dropout(0.3))
    nn.add(layers.SeparableConv2D(256, (3, 3), padding = 'same',
                                  activation = 'relu'))
    # Shape: (23, 116, 256)
    nn.add(layers.BatchNormalization())
    nn.add(layers.Dropout(0.3))
    nn.add(layers.SeparableConv2D(256, (3, 3), padding = 'same',
                                  activation = 'relu'))
    # Shape: (21, 114, 256)
    nn.add(layers.BatchNormalization())
    nn.add(layers.MaxPooling2D((2, 2)))
    # Shape: (10, 57, 256)
    nn.add(layers.Dropout(0.3))

#     nn.add(layers.SeparableConv2D(512, (3, 3), padding = 'same',
#                                   activation = 'relu'))
#     # Shape: (8, 55, 512)
#     nn.add(layers.BatchNormalization())
#     nn.add(layers.SeparableConv2D(512, (3, 3), padding = 'same',
#                                   activation = 'relu'))
#     # Shape: (6, 53, 512)
#     nn.add(layers.BatchNormalization())
#     nn.add(layers.MaxPooling2D((2, 2)))
#     # Shape: (3, 26, 512)
#     nn.add(layers.Dropout(0.3))

    nn.add(layers.SeparableConv2D(512, (3, 3), padding = 'same',
                                  activation = 'relu'))
    # Shape: (1, 24, 512)
    nn.add(layers.BatchNormalization())
    nn.add(layers.SeparableConv2D(512, (3, 3), padding = 'same',
                                  activation = 'relu'))
    nn.add(layers.BatchNormalization())
    nn.add(layers.GlobalAveragePooling2D())

    nn.add(layers.Dense(41, activation = 'softmax'))
    return nn


# Get data
# input_path = 'AudioTaggerData/'
input_path = 'drive/My Drive/AudioTaggerData/'

train_files_path = input_path + 'FSDKaggle2018.audio_train'
test_files_path = input_path + 'FSDKaggle2018.audio_test'
train_csv_path = (input_path +
                  'FSDKaggle2018.meta/train_post_competition.csv')
test_csv_path = (input_path +
                 'FSDKaggle2018.meta/' +
                 'test_post_competition_scoring_clips.csv')


def get_train_data():
#     data = np.ndarray((9474, 64, 32))
    data = np.ndarray((9474, 256, 128))

#     data_len, data = 9474, []
    data_len, time_len, freq = 9474, 256, 128

#     data_len, data = 3, []
    with open(input_path + 'Audio.train', 'r') as data_f:
        for i in range(data_len):
#             sample = []
#             time_step = [float(elem) for elem in
#                          data_f.readline().split()]
#             print(time_step)
            for j in range(time_len):
#             while time_step != []:
#                 sample.append(np.array(time_step))
                
    
                time_step = [float(elem) for elem in
                             data_f.readline().split()]
#                 sample.append(time_step)
                
                if len(time_step) < freq:
                    rest = freq - len(time_step)
                    time_step += [-100.0 for x in range(rest)]
    
                for k in range(freq):
#                     if (len(time_step) != 32):
#                         print('BAD time step len:', i)
#                     if (i >= 9474 or j >= 64 or k >= 32):
#                         print('OOB error:')
#                         print(i,j,k)
#                     print(len(time_step))
                    data[i][j][k] = time_step[k]
            
#                 print(time_step)
                # print('line')
                # print(line)
#                 sample.append(np.array(time_step))
#             data.append(np.array(sample))
#             data.append(sample)

            data_f.readline()

#             print('DATA:')
#             print(data)

#     return np.array(data)
    return data

def get_test_data():
    data = np.ndarray((1570, 256, 128))
    data_len = 1570

#     data_len, data = 3, []
    with open(input_path + 'Audio.test', 'r') as data_f:
        for i in range(data_len):

            for j in range(256):
    
                time_step = [float(elem) for elem in
                             data_f.readline().split()]
#                 sample.append(time_step)
                
                if len(time_step) < 128:
                    rest = 128 - len(time_step)
                    time_step += [-100.0 for x in range(rest)]
    
                for k in range(128):
                    data[i][j][k] = time_step[k]

            data_f.readline()

    return data


def get_train_labels():
    labels = np.ndarray((9474, 41))
#     labels_len, labels = 9474, []
    labels_len = 9474

#     labels_len, labels = 3, []
    with open(input_path + 'Labels.train', 'r') as label_f:
        for i in range(labels_len):
            label = [float(elem) for elem in label_f.readline().split()]
#             labels.append(label)

            if len(label) < 41:
                    rest = 41 - len(label)
                    label += [0.0 for x in range(rest)]
            
            for j in range(41):
                labels[i][j] = label[j]

#     return np.array(labels)
    return labels

def get_test_labels():
    labels = np.ndarray((1570, 41))
#     labels_len, labels = 9474, []
    labels_len = 1570

#     labels_len, labels = 3, []
    with open(input_path + 'Labels.test', 'r') as label_f:
        for i in range(labels_len):
            label = [float(elem) for elem in label_f.readline().split()]
#             labels.append(label)

            if len(label) < 41:
                    rest = 41 - len(label)
                    label += [0.0 for x in range(rest)]
            
            for j in range(41):
                labels[i][j] = label[j]

#     return np.array(labels)
    return labels

# 1. Create training data (log mel spectrograms)
# dummy_samples = 10
# Num train samples = 9474
# Num test samples = 1570



# UNDO
# train_data = get_train_data()
# train_labels = get_train_labels()

# test_data = get_test_data()
# test_labels = get_test_labels()
test_data = get_data(test_files_path, training = False)
test_labels = get_labels(test_files_path, training = False)

#UNDO
# train_samples, test_samples = 9474, 1570
# max_timesteps = train_data.shape[1]
# num_freq = train_data.shape[2]


0001 / 1600 | drive/My Drive/AudioTaggerData/FSDKaggle2018.audio_test/40f595b9.wav


/usr/local/lib/python3.6/dist-packages/librosa/core/time_frequency.py:1006: RuntimeWarning: divide by zero encountered in log10
  - 0.5 * np.log10(f_sq + const[3]))


0002 / 1600 | drive/My Drive/AudioTaggerData/FSDKaggle2018.audio_test/a9a0f11e.wav


/usr/local/lib/python3.6/dist-packages/librosa/core/time_frequency.py:1006: RuntimeWarning: divide by zero encountered in log10
  - 0.5 * np.log10(f_sq + const[3]))


0003 / 1600 | drive/My Drive/AudioTaggerData/FSDKaggle2018.audio_test/9958521a.wav


...

1309 / 1600 | drive/My Drive/AudioTaggerData/FSDKaggle2018.audio_test/520dd11d.wav


/usr/local/lib/python3.6/dist-packages/librosa/core/time_frequency.py:1006: RuntimeWarning: divide by zero encountered in log10
  - 0.5 * np.log10(f_sq + const[3]))


1310 / 1600 | drive/My Drive/AudioTaggerData/FSDKaggle2018.audio_test/43723640.wav


/usr/local/lib/python3.6/dist-packages/librosa/core/time_frequency.py:1006: RuntimeWarning: divide by zero encountered in log10
  - 0.5 * np.log10(f_sq + const[3]))


1311 / 1600 | drive/My Drive/AudioTaggerData/FSDKaggle2018.audio_test/2f1c2612.wav


In [18]:
mean = train_data.mean(axis = 0)
# print('Mean of sample 1:')
# print(mean[0])
train_data -= mean
std = train_data.std(axis = 0)
train_data /= std

# ONCE I HAVE TEST DATA
test_data -= mean
test_data /= std

# Use getter methods here
# dummy_train_data = np.random.random((dummy_samples, dummy_max_timesteps,
#                                      dummy_num_freq, 1))
# # Categorical labels
# dummy_train_labels = np.zeros((10, 41))
# for i, label in enumerate(dummy_train_labels):
#     label[i] = 1
# print(dummy_train_labels)

model = make_model((max_timesteps, num_freq, 1))
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

k = 4
num_val = len(train_data) // k
num_train = len(train_labels) - num_val
all_val_acc_histories, all_val_loss_histories = [], []
for x in range(k):
    val_data = train_data[x * num_val: (x + 1) * num_val]
    val_labels = train_labels[x * num_val: (x + 1) * num_val]

    partial_train_data = np.concatenate(
        [train_data[: x * num_val], train_data[(x + 1) * num_val:]],
        axis = 0)
    partial_train_labels = np.concatenate(
        [train_labels[: x * num_val],
         train_labels[(x + 1) * num_val:]],
        axis = 0)
    
    partial_train_data = partial_train_data.reshape(
            (partial_train_data.shape[0], 
             partial_train_data.shape[1], 
             partial_train_data.shape[2], 
             1))
    
    val_data = val_data.reshape(
            (val_data.shape[0], 
             val_data.shape[1], 
             val_data.shape[2], 
             1))
    
    hst = model.fit(partial_train_data, partial_train_labels, batch_size = 1024,
                    epochs = 10, validation_data = (val_data, val_labels),)

    hst = hst.history
    all_val_loss_histories.append(hst['val_loss'])
    all_val_acc_histories.append(hst['val_acc'])

avg_val_loss_hst = np.mean(all_val_loss_histories, axis = 0)
avg_val_acc_hst = np.mean(all_val_acc_histories, axis = 0)

best_loss, best_acc, prev_acc, best_epoch = None, None, None, 0

acc_increased = True
for i in range(10):
    print(avg_val_acc_hst[i], '/', avg_val_loss_hst[i])

    if prev_acc is not None and avg_val_acc_hst[i] < prev_acc:
        acc_increased = False
    prev_acc = avg_val_loss_hst[i]

    if (best_acc is None or avg_val_acc_hst[i] > best_acc and
            acc_increased):
        best_acc = avg_val_acc_hst[i]
        best_loss = avg_val_loss_hst[i]
        best_epoch = i + 1

print('Best val loss:', best_loss, '& with acc:', best_acc, 'at epoch:',
      str(best_epoch))

model.save('2DCNN.h5')


Mean of sample 1:
[-86.62562035 -80.24301205 -76.96972696 -74.24042989 -72.61073198
 -72.02183324 -71.96739275 -71.94178303 -72.11115085 -72.31134021
 -72.43132402 -73.15902811 -73.39463413 -73.82801204 -74.54069714
 -75.03079402 -75.5396978  -76.29402385 -76.83100721 -77.34660071
 -77.61326341 -78.22247951 -78.70824379 -79.37451051 -80.45232453
 -81.33338928 -82.41725067 -83.25649102 -84.30028947 -85.23726603
 -86.02354532 -86.80838875]
Train on 7106 samples, validate on 2368 samples
Epoch 1/10
7106/7106 [==============================] - 9s 1ms/step - loss: 3.4839 - acc: 0.0948 - val_loss: 3.5066 - val_acc: 0.1035
Epoch 2/10
7106/7106 [==============================] - 6s 838us/step - loss: 3.1340 - acc: 0.1714 - val_loss: 3.1900 - val_acc: 0.1660
Epoch 3/10
7106/7106 [==============================] - 6s 843us/step - loss: 2.9191 - acc: 0.2118 - val_loss: 3.1819 - val_acc: 0.1959
Epoch 4/10
7106/7106 [==============================] - 6s 832us/step - loss: 2.7378 - acc: 0.2518 - val

In [19]:


def make_combined_CNN_RNN_model(input_shape):
    nn = models.Sequential()
    nn.add(layers.SeparableConv1D(64, 5, activation = 'relu',
                                  input_shape = (None, input_shape[-1])))
    nn.add(layers.BatchNormalization())
    nn.add(layers.SeparableConv1D(64, 5, activation = 'relu'))
    nn.add(layers.BatchNormalization())
    nn.add(layers.MaxPooling1D(3))
    nn.add(layers.Dropout(0.3))

    nn.add(layers.SeparableConv1D(128, 5, activation = 'relu'))
    nn.add(layers.BatchNormalization())
    nn.add(layers.SeparableConv1D(128, 5, activation = 'relu'))
    nn.add(layers.BatchNormalization())
    # nn.add(layers.MaxPooling1D(3))
    # nn.add(layers.Dropout(0.3))
    #
    # nn.add(layers.SeparableConv1D(512, 5, activation = 'relu'))
    # nn.add(layers.BatchNormalization())
    # nn.add(layers.SeparableConv1D(512, 5, activation = 'relu'))
    # nn.add(layers.BatchNormalization())

    nn.add(layers.Bidirectional(layers.LSTM(128, dropout = 0.3,
                                            recurrent_dropout = 0.3,
                                            return_sequences = True)))
    nn.add(layers.Bidirectional(layers.LSTM(128, dropout = 0.3,
                                            recurrent_dropout = 0.3,
                                            return_sequences = True)))
    nn.add(layers.Bidirectional(layers.LSTM(128, dropout = 0.3,
                                            recurrent_dropout = 0.3,
                                            return_sequences = True)))
    nn.add(layers.Bidirectional(layers.LSTM(128, dropout = 0.3,
                                            recurrent_dropout = 0.3)))

    nn.add(layers.Dense(41, activation = 'softmax'))

    return nn

model = make_combined_CNN_RNN_model((max_timesteps, num_freq))
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

k = 4
num_val = len(train_data) // k
num_train = len(train_labels) - num_val
all_val_acc_histories, all_val_loss_histories = [], []
for x in range(k):
    val_data = train_data[x * num_val: (x + 1) * num_val]
    val_labels = train_labels[x * num_val: (x + 1) * num_val]

    partial_train_data = np.concatenate(
        [train_data[: x * num_val], train_data[(x + 1) * num_val:]],
        axis = 0)
    partial_train_labels = np.concatenate(
        [train_labels[: x * num_val],
         train_labels[(x + 1) * num_val:]],
        axis = 0)
    
    hst = model.fit(partial_train_data, partial_train_labels, batch_size = 1024,
                    epochs = 10, validation_data = (val_data, val_labels),)

    hst = hst.history
    all_val_loss_histories.append(hst['val_loss'])
    all_val_acc_histories.append(hst['val_acc'])

avg_val_loss_hst = np.mean(all_val_loss_histories, axis = 0)
avg_val_acc_hst = np.mean(all_val_acc_histories, axis = 0)

best_loss, best_acc, prev_acc, best_epoch = None, None, None, 0

acc_increased = True
for i in range(10):
    print(avg_val_acc_hst[i], '/', avg_val_loss_hst[i])

    if prev_acc is not None and avg_val_acc_hst[i] < prev_acc:
        acc_increased = False
    prev_acc = avg_val_loss_hst[i]

    if (best_acc is None or avg_val_acc_hst[i] > best_acc and
            acc_increased):
        best_acc = avg_val_acc_hst[i]
        best_loss = avg_val_loss_hst[i]
        best_epoch = i + 1

print('Best val loss:', best_loss, '& with acc:', best_acc, 'at epoch:',
      str(best_epoch))

model.save('1DCNN_RNN.h5')

Train on 7106 samples, validate on 2368 samples
Epoch 1/10
7106/7106 [==============================] - 13s 2ms/step - loss: 3.5468 - acc: 0.0764 - val_loss: 3.3753 - val_acc: 0.1199
Epoch 2/10
7106/7106 [==============================] - 2s 217us/step - loss: 3.2932 - acc: 0.1203 - val_loss: 3.2199 - val_acc: 0.1410
Epoch 3/10
7106/7106 [==============================] - 2s 215us/step - loss: 3.1673 - acc: 0.1503 - val_loss: 3.1845 - val_acc: 0.1508
Epoch 4/10
7106/7106 [==============================] - 2s 214us/step - loss: 3.0714 - acc: 0.1689 - val_loss: 3.1538 - val_acc: 0.1715
Epoch 5/10
7106/7106 [==============================] - 2s 216us/step - loss: 2.9971 - acc: 0.1797 - val_loss: 3.1689 - val_acc: 0.1660
Epoch 6/10
7106/7106 [==============================] - 2s 218us/step - loss: 2.9317 - acc: 0.1929 - val_loss: 3.0140 - val_acc: 0.1934
Epoch 7/10
7106/7106 [==============================] - 2s 215us/step - loss: 2.9041 - acc: 0.1990 - val_loss: 2.9383 - val_acc: 0.2086
E

In [28]:
from keras.applications import DenseNet121

def make_dense_net_model(conv_base):
    nn = models.Sequential()
    nn.add(conv_base)
    nn.add(layers.Dense(41, activation = 'softmax'))
    return nn

dn_base = DenseNet121(include_top = False,
                      input_shape = (max_timesteps, num_freq, 3),
                      pooling = 'avg')
dn_base.trainable = True
# print(dn_base.summary())
# Fine-tuning
set_trainable = False
for layer in dn_base.layers:
    if layer.name == 'conv5_block13_0_bn':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

model = make_dense_net_model(dn_base)
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

k = 4
num_val = len(train_data) // k
num_train = len(train_labels) - num_val
all_val_acc_histories, all_val_loss_histories = [], []
for x in range(k):
    val_data = train_data[x * num_val: (x + 1) * num_val]
    val_labels = train_labels[x * num_val: (x + 1) * num_val]

    partial_train_data = np.concatenate(
        [train_data[: x * num_val], train_data[(x + 1) * num_val:]],
        axis = 0)
    partial_train_labels = np.concatenate(
        [train_labels[: x * num_val],
         train_labels[(x + 1) * num_val:]],
        axis = 0)
    
    partial_train_data = partial_train_data.reshape(
            (partial_train_data.shape[0], 
             partial_train_data.shape[1], 
             partial_train_data.shape[2], 
             1))
    
    partial_train_data = np.repeat(partial_train_data, 3, axis=3)
    
    val_data = val_data.reshape(
            (val_data.shape[0], 
             val_data.shape[1], 
             val_data.shape[2], 
             1))
    
    val_data = np.repeat(val_data, 3, axis=3)
#     print('Val Data shape:')
#     print(val_data.shape)

    
    hst = model.fit(partial_train_data, partial_train_labels, batch_size = 1024,
                    epochs = 10, validation_data = (val_data, val_labels),)

    hst = hst.history
    all_val_loss_histories.append(hst['val_loss'])
    all_val_acc_histories.append(hst['val_acc'])

avg_val_loss_hst = np.mean(all_val_loss_histories, axis = 0)
avg_val_acc_hst = np.mean(all_val_acc_histories, axis = 0)

best_loss, best_acc, prev_acc, best_epoch = None, None, None, 0

acc_increased = True
for i in range(10):
    print(avg_val_acc_hst[i], '/', avg_val_loss_hst[i])

    if prev_acc is not None and avg_val_acc_hst[i] < prev_acc:
        acc_increased = False
    prev_acc = avg_val_loss_hst[i]

    if (best_acc is None or avg_val_acc_hst[i] > best_acc and
            acc_increased):
        best_acc = avg_val_acc_hst[i]
        best_loss = avg_val_loss_hst[i]
        best_epoch = i + 1

print('Best val loss:', best_loss, '& with acc:', best_acc, 'at epoch:',
      str(best_epoch))

model.save('DenseNetModel.h5')

Train on 7106 samples, validate on 2368 samples
Epoch 1/10
7106/7106 [==============================] - 46s 7ms/step - loss: 3.8703 - acc: 0.1057 - val_loss: 11.5388 - val_acc: 0.0498
Epoch 2/10
7106/7106 [==============================] - 2s 339us/step - loss: 2.7379 - acc: 0.2831 - val_loss: 11.3960 - val_acc: 0.0465
Epoch 3/10
7106/7106 [==============================] - 2s 337us/step - loss: 2.2438 - acc: 0.4021 - val_loss: 11.2043 - val_acc: 0.0460
Epoch 4/10
7106/7106 [==============================] - 2s 340us/step - loss: 1.8462 - acc: 0.5250 - val_loss: 10.9872 - val_acc: 0.0481
Epoch 5/10
7106/7106 [==============================] - 2s 341us/step - loss: 1.5243 - acc: 0.6288 - val_loss: 11.6239 - val_acc: 0.0460
Epoch 6/10
7106/7106 [==============================] - 2s 343us/step - loss: 1.2511 - acc: 0.7181 - val_loss: 11.2657 - val_acc: 0.0439
Epoch 7/10
7106/7106 [==============================] - 2s 340us/step - loss: 1.0402 - acc: 0.7812 - val_loss: 11.6753 - val_acc: 0

In [29]:


def make_1DCNN_model(input_shape):
    nn = models.Sequential()
    nn.add(layers.SeparableConv1D(64, 7, activation = 'relu',
                                  input_shape = (None, input_shape[-1])))
    nn.add(layers.BatchNormalization())
    nn.add(layers.SeparableConv1D(64, 7, activation = 'relu'))
    nn.add(layers.BatchNormalization())
    nn.add(layers.MaxPooling1D(5))
    nn.add(layers.Dropout(0.3))

    nn.add(layers.SeparableConv1D(128, 7, activation = 'relu'))
    nn.add(layers.BatchNormalization())
    nn.add(layers.SeparableConv1D(128, 7, activation = 'relu'))
    nn.add(layers.BatchNormalization())
    nn.add(layers.MaxPooling1D(5))
    nn.add(layers.Dropout(0.3))

    nn.add(layers.SeparableConv1D(512, 7, activation = 'relu'))
    nn.add(layers.BatchNormalization())
    nn.add(layers.SeparableConv1D(512, 7, activation = 'relu'))
    nn.add(layers.BatchNormalization())
    nn.add(layers.GlobalAveragePooling1D())

    nn.add(layers.Dense(41, activation = 'softmax'))

    return nn

model = make_1DCNN_model((max_timesteps, num_freq))
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

k = 4
num_val = len(train_data) // k
num_train = len(train_labels) - num_val
all_val_acc_histories, all_val_loss_histories = [], []
for x in range(k):
    val_data = train_data[x * num_val: (x + 1) * num_val]
    val_labels = train_labels[x * num_val: (x + 1) * num_val]

    partial_train_data = np.concatenate(
        [train_data[: x * num_val], train_data[(x + 1) * num_val:]],
        axis = 0)
    partial_train_labels = np.concatenate(
        [train_labels[: x * num_val],
         train_labels[(x + 1) * num_val:]],
        axis = 0)
    
    hst = model.fit(partial_train_data, partial_train_labels, batch_size = 1024,
                    epochs = 10, validation_data = (val_data, val_labels),)

    hst = hst.history
    all_val_loss_histories.append(hst['val_loss'])
    all_val_acc_histories.append(hst['val_acc'])

avg_val_loss_hst = np.mean(all_val_loss_histories, axis = 0)
avg_val_acc_hst = np.mean(all_val_acc_histories, axis = 0)

best_loss, best_acc, prev_acc, best_epoch = None, None, None, 0

acc_increased = True
for i in range(10):
    print(avg_val_acc_hst[i], '/', avg_val_loss_hst[i])

    if prev_acc is not None and avg_val_acc_hst[i] < prev_acc:
        acc_increased = False
    prev_acc = avg_val_loss_hst[i]

    if (best_acc is None or avg_val_acc_hst[i] > best_acc and
            acc_increased):
        best_acc = avg_val_acc_hst[i]
        best_loss = avg_val_loss_hst[i]
        best_epoch = i + 1

print('Best val loss:', best_loss, '& with acc:', best_acc, 'at epoch:',
      str(best_epoch))

model.save('1DCNN.h5')


Train on 7106 samples, validate on 2368 samples
Epoch 1/10


InvalidArgumentError: ignored

In [0]:
def make_RNN_model(input_shape):
    nn = models.Sequential()
    nn.add(layers.Bidirectional(layers.LSTM(64, dropout = 0.3,
                                            recurrent_dropout = 0.3,
                                            return_sequences = True,
                                            input_shape =
                                            (None, input_shape[-1]))))
    nn.add(layers.Bidirectional(layers.LSTM(64, dropout = 0.3,
                                            recurrent_dropout = 0.3,
                                            return_sequences = True)))
    nn.add(layers.Bidirectional(layers.LSTM(64, dropout = 0.3,
                                            recurrent_dropout = 0.3,
                                            return_sequences = True)))
    nn.add(layers.Bidirectional(layers.LSTM(64, dropout = 0.3,
                                            recurrent_dropout = 0.3)))
    nn.add(layers.Dense(41, activation = 'softmax'))

    return nn


model = make_RNN_model((max_timesteps, num_freq))
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

k = 4
num_val = len(train_data) // k
num_train = len(train_labels) - num_val
all_val_acc_histories, all_val_loss_histories = [], []
for x in range(k):
    val_data = train_data[x * num_val: (x + 1) * num_val]
    val_labels = train_labels[x * num_val: (x + 1) * num_val]

    partial_train_data = np.concatenate(
        [train_data[: x * num_val], train_data[(x + 1) * num_val:]],
        axis = 0)
    partial_train_labels = np.concatenate(
        [train_labels[: x * num_val],
         train_labels[(x + 1) * num_val:]],
        axis = 0)
    
    hst = model.fit(partial_train_data, partial_train_labels, batch_size = 1024,
                    epochs = 10, validation_data = (val_data, val_labels),)

    hst = hst.history
    all_val_loss_histories.append(hst['val_loss'])
    all_val_acc_histories.append(hst['val_acc'])

avg_val_loss_hst = np.mean(all_val_loss_histories, axis = 0)
avg_val_acc_hst = np.mean(all_val_acc_histories, axis = 0)

best_loss, best_acc, prev_acc, best_epoch = None, None, None, 0

acc_increased = True
for i in range(10):
    print(avg_val_acc_hst[i], '/', avg_val_loss_hst[i])

    if prev_acc is not None and avg_val_acc_hst[i] < prev_acc:
        acc_increased = False
    prev_acc = avg_val_loss_hst[i]

    if (best_acc is None or avg_val_acc_hst[i] > best_acc and
            acc_increased):
        best_acc = avg_val_acc_hst[i]
        best_loss = avg_val_loss_hst[i]
        best_epoch = i + 1

print('Best val loss:', best_loss, '& with acc:', best_acc, 'at epoch:',
      str(best_epoch))

model.save('RNN.h5')


In [0]:
# Reorganize all data as sets of flat vectors of 0-1.0 values
# Shape (..., 28, 28, 1)

test_data_2D = test_data.reshape(
               (test_data.shape[0], 
                test_data.shape[1], 
                test_data.shape[2], 
                1))
    
test_data_2D_3chan = np.repeat(test_data_2D, 3, axis=3)
    

cnn2D = models.load_model('2DCNN.h5')
rnn = models.load_model('RNN.h5')
cnn1D = models.load_model('1DCNN.h5')
cnn_rnn = models.load_model('1DCNN_RNN.h5')
dense_net = models.load_model('DenseNetModel.h5')

cnn2D_preds = cnn2D.predict(test_data_2D)
rnn_preds = rnn.predict(test_data)
cnn1D_preds = cnn1D.predict(test_data)
cnn_rnn_preds = cnn_rnn.predict(test_data)
dense_net_preds = dense_net.predict(test_data_2D_3chan)

final_preds = 0.2 * (cnn2D_preds + rnn_preds + cnn1D_preds + 
                     cnn_rnn_preds + dense_net_preds)

with open('TestTags.txt', 'w') as ttf:
    for i, res in enumerate(final_preds):
        ttf.write(i+')', unique_labels[res.index(1.)])


# Prompt for user to enter a wav filename then predicts??

# batch_size = 128
# results = nn.predict(test_data, batch_size = batch_size)
# for result in results:
#     res = np.argmax(result)
#     print(res)